# Networks from OpenStreetMap


- crée un network à partir d'un fichier .osm et des trajectoires de véhiculess
- ajoute un flux de voiture sur les routes
- personnalise un Environnement pour le RL
- integrate it and run it in Flow.


In [1]:
from flow.core.params import VehicleParams
from flow.core.params import NetParams, SumoCarFollowingParams,SumoLaneChangeParams 
from flow.core.params import InitialConfig
from flow.core.params import EnvParams
from flow.core.params import SumoParams
from flow.controllers import RLController, IDMController
from flow.networks.IssyOSMNetwork import IssyOSMNetwork
from flow.networks import Network
from flow.core.params import InFlows

## Importation du network d'Issy

On vérifie si IssyOSMNetwork est bien importé.

In [2]:
from flow.networks.IssyOSMNetwork import ADDITIONAL_NET_PARAMS, EDGES_DISTRIBUTION

print(ADDITIONAL_NET_PARAMS)
print(EDGES_DISTRIBUTION)

{'speed_limit': 50}
['-100822066', '4794817', '4783299#0', '155558218']


## Ajoute les flux de voiture

`IDMController` : The Intelligent Driver Model is a car-following model specifying vehicle dynamics by a differential equation for acceleration $\dot{v}$.

`RLController` : a trainable autuonomous vehicle whose actions are dictated by an RL agent. 

In [3]:
import MinicityRouter
from flow.controllers import ContinuousRouter

In [4]:
vehicles = VehicleParams()
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="right_of_way"),
             lane_change_params=SumoLaneChangeParams(
                 lane_change_mode=2722)
             )
vehicles.add("rl",
             acceleration_controller=(IDMController, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="right_of_way"),
             lane_change_params=SumoLaneChangeParams(
                 lane_change_mode=2722),
             color="cyan")

- `vehs_per_hour`: nombre de vehicule par heure, uniformément espacés. Par exemple, comme il y a $60 \times 60 = 3600$ secondes dans une heure, le parametre $\frac{3600}{5}=720$ va faire rentrer des vehicules dans le network toutes les $5$ secondes.

- `probability`: c'est la probabilité qu'un véhicule entre dans le network toutes les secondes. Par exemple, si on la fixe à $0.2$, alors chaque seconde de la simulation un véhicule aura $\frac{1}{5}$ chance d'entrer dans le network

- `period`: C'est le temps en secondes entre 2 véhicules qui sont insérés. Par exemple, le fixer à $5$ ferait rentrer des véhicules dans le network toutes les $5$ secondes (ce qui équivaut à mettre `vehs_per_hour` à $720$).

<font color='red'>
$\rightarrow$ Exactement 1 seul de ces 3 paramètres doit être configurer !
</font>

In [5]:
inflow = InFlows()
 
for edge in EDGES_DISTRIBUTION:
    inflow.add(veh_type = "human", 
               edge = edge,
               probability = 0.01,
               name = "human",
               depart_lane = 0,
               depart_speed = 7)

## Personnalise un Environnement pour le RL

plus de méthodes sur : http://berkeleyflow.readthedocs.io/en/latest/

In [6]:
# import the base environment class
from flow.envs import Env
from gym.spaces.box import Box
from gym.spaces import Tuple
from flow.core.experiment import Experiment
from myenv import myEnv
import numpy as np

## Lance une simulation

In [7]:
HORIZON = 5000

In [8]:
ADDITIONAL_ENV_PARAMS = {"max_accel": 2, "max_decel": 2}

# SUMO PARAM
sumo_params = SumoParams(sim_step=0.2, 
                         print_warnings=False, 
                         render=True, 
                         emission_path='/home/lino/Documents/DQN_CIL4SYS/data',
                         restart_instance=True,
                         overtake_right=True) #, ,  )

# ENVIRONMENT PARAM
env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS, horizon=HORIZON)

# NETWORK PARAM
net_params = NetParams(inflows=inflow, osm_path='/home/lino/Documents/flow/flow/networks/issy.osm')

# NETWORK
network = IssyOSMNetwork

# INITIAL CONFIG
initial_config = InitialConfig(edges_distribution=EDGES_DISTRIBUTION) # spacing='random'

# dictionnaire FLOW_PARAM
flow_params = dict( exp_tag  = 'ISSY_trial01',
                    env_name = myEnv,  # using my new environment for the simulation
                    network  = network,
                    simulator= 'traci',
                    sim      = sumo_params,
                    env      = env_params,
                    net      = net_params,
                    veh      = vehicles,
                    initial  = initial_config)

# create EXPERIMENT with class created
exp = Experiment(flow_params)

# RUN SIMULATION SUMO
_ = exp.run(1, convert_to_csv=True)


/home/lino/anaconda3/envs/flow/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/lino/anaconda3/envs/flow/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Round 0, return: nan
Average, std returns: nan, nan
Average, std velocities: nan, nan
Average, std outflows: 0.0, 0.0
Total time: 57.428839445114136
steps/second: 181.41846882853324


KeyboardInterrupt: 

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/home/lino/Documents/DQN_CIL4SYS/data/ISSY_trial01_20200507-1209401588846180.5759099-emission.csv')

In [ ]:
df

In [ ]:
df_clean = df.drop(columns=['y','electricity', 'type', 'eclass', 
                            'NOx', 'HC','x','route','relative_position','noise','angle','PMx'])

In [ ]:
df_clean

In [ ]:
len(df_clean.drop_duplicates(['id']))